In [1]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # use GPU
# Using GPU during inference has deterministic results (same as CPU)

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs import utils, mnist32_cnn
from libs.constants import model_seeds

In [2]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
# Prepare dataset
# Combine test and train images together into one dataset
DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0  

all_images =np.concatenate([train_images, test_images], axis=0)
all_labels =np.concatenate([train_labels, test_labels], axis=0)
all_images = np.expand_dims(all_images, axis=-1)

# resize the input shape , i.e. old shape: 28, new shape: 32
image_x_size = 32
image_y_size = 32
all_images = tf.image.resize(all_images, [image_x_size, image_y_size]) 

In [4]:
model_type = "mnist32-cnn_1024_256_64"
model_seed = model_seeds[0]

In [5]:
# Get model
model_instance = model_type + "-" + str(model_seed)
dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)

model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_meta_type / model_type)
model_filename = model_instance + ".h5"
model_file = pathlib.Path(model_folder/ model_filename)

In [6]:
images = all_images[0:15]
# image = tf.expand_dims(all_images[0],axis=0)

In [7]:
# Load model
model = tf.keras.models.load_model(model_file)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 29, 29, 32)        544       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 fc_0 (Dense)                (None, 1024)              6423552   
                                                                 
 fc_1 (Dense)                (None, 256)               262400    
                                                                 
 fc_2 (Dense)                (None, 64)                16448     
                                                        

In [8]:
conv_out = model.layers[0](images)

In [9]:
pool_out = model.layers[1](
                model.layers[0](images))

In [10]:
flat_out = model.layers[2](
                model.layers[1](
                    model.layers[0](images)))

In [11]:
fc_0_out = model.layers[3](
                model.layers[2](
                    model.layers[1](
                        model.layers[0](images))))

In [12]:
fc_1_out = model.layers[4](
                model.layers[3](
                  model.layers[2](
                      model.layers[1](
                          model.layers[0](images)))))

In [13]:
fc_2_out = model.layers[5](
              model.layers[4](
                  model.layers[3](
                      model.layers[2](
                          model.layers[1](
                              model.layers[0](images))))))

In [14]:
op_layer_out = model.layers[6](
                  model.layers[5](
                      model.layers[4](
                          model.layers[3](
                              model.layers[2](
                                  model.layers[1](
                                      model.layers[0](images)))))))

In [15]:
tf.reduce_sum(op_layer_out - model(images))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [16]:
op_layer_out

<tf.Tensor: shape=(15, 10), dtype=float32, numpy=
array([[2.71675174e-12, 3.35802480e-12, 6.55135043e-11, 1.64698791e-02,
        2.62287544e-12, 9.83529866e-01, 8.16055348e-11, 4.47783616e-11,
        1.03576543e-08, 1.78205411e-07],
       [9.99990821e-01, 9.73690589e-07, 7.60873536e-06, 2.72391198e-09,
        2.23365770e-08, 1.08090892e-09, 1.82814304e-07, 1.20004643e-07,
        6.05420025e-09, 2.56503085e-07],
       [7.64120074e-08, 1.08786495e-04, 5.45928851e-05, 3.62876762e-07,
        9.99743521e-01, 1.86826782e-07, 8.30925018e-08, 8.59544234e-05,
        9.79177344e-08, 6.15908402e-06],
       [8.56766533e-08, 9.99975681e-01, 2.30086272e-07, 2.04455217e-10,
        9.00513623e-06, 2.33917081e-08, 4.04213303e-08, 1.45030317e-05,
        3.65521089e-07, 8.12035150e-09],
       [1.05059872e-08, 1.07096344e-06, 2.58097552e-05, 3.54660478e-05,
        1.58940081e-03, 1.93853020e-06, 1.29515912e-10, 1.49401691e-04,
        1.65589190e-05, 9.98180389e-01],
       [9.41910816e-09, 6

In [19]:
fc_2_out

<tf.Tensor: shape=(15, 64), dtype=float32, numpy=
array([[ 0.        ,  0.        ,  7.029243  ,  0.5471495 ,  2.5350912 ,
         1.6151757 ,  4.0116043 ,  0.        ,  5.3669305 ,  0.        ,
         3.7226794 , 10.914504  ,  2.089747  ,  0.        ,  0.        ,
         1.9117383 ,  6.807363  ,  7.7262974 ,  4.6767454 ,  0.        ,
         0.        ,  0.        ,  2.291761  ,  6.2492948 ,  5.335433  ,
         7.646982  ,  0.        ,  8.407822  ,  6.723912  ,  0.        ,
         7.404195  ,  0.02817028,  0.        ,  5.5928273 ,  4.4628625 ,
         0.        ,  8.510649  ,  1.3339181 ,  3.145214  ,  0.        ,
         0.9278116 ,  7.222084  ,  3.2029016 ,  0.        ,  1.6681744 ,
         0.        ,  0.        ,  1.315224  ,  5.17088   ,  8.520081  ,
         0.        ,  3.4152188 ,  1.2961205 ,  3.1464596 ,  0.        ,
         4.2674    ,  2.1508512 ,  4.743614  ,  0.        ,  1.8361688 ,
         9.36847   ,  6.500458  ,  4.6127214 ,  4.4707108 ],
       [ 2.72